In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head(2)

In [ ]:
credits.head(1)

In [ ]:
# credits.head(1)['cast'].values

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies = pd.merge(movies,credits,on='title')

In [ ]:
movies.shape

In [ ]:
movies.head(2)

In [ ]:
type(movies)

In [ ]:
movies['original_language'].value_counts()
# here english is the most common langauge so it dosen't matter atmost about the recommendation

In [ ]:
movies.info()

In [ ]:
# required columns for the features labelling(string format are basically taken so that the tags column can be extracted)
# 1. genres
# 2. id
# 3. title
# 4. overview
# 5. cast
# 6. crew
# 7. keyowrds

movies = movies[['movie_id','title','keywords','genres','overview','cast','crew']]

In [ ]:
# final dataframe to work on further
movies.head(2)

In [ ]:
# chceking if any nul values are presnt in the dataset or not and removing the null values
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
# all the null values are being removed 
movies.isnull().sum()

In [ ]:
# 0 indicates that no duplicate data are present in the dataset
movies.duplicated().sum()

In [ ]:
# This is in a weird format ex: This is a list of dictionary And we want a list only which can be further used for prediction
movies.iloc[0]['genres']

In [ ]:
# this will make the string as a list of dictionary 
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
def convert(geners):
    L = []
    for i in ast.literal_eval(geners):
        L.append(i['name'])
    return L

In [ ]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head(2)

In [ ]:
movies.iloc[0]['keywords']

In [ ]:
movies.iloc[0]['cast']

In [ ]:
def get_three(geners):
    L = []
    counter = 0
    for i in ast.literal_eval(geners):
        if counter!= 3: 
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(get_three)

In [ ]:
movies.iloc[0]['cast']

In [ ]:
movies.head(3)

In [ ]:
movies.iloc[0]['crew']

In [ ]:
# get the director
def get_director(geners):
    L = []
    for i in ast.literal_eval(geners):
        if i['job'] == "Director": 
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(get_director)

In [ ]:
movies.head(2)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(3)

In [ ]:
# as the data is ready to feed but it consists of some minor issues 
# issues are that if a words is like sam mendes and sam worthington the model will get confused so we need to handle this by removing the space in Between the words
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head(3)

In [ ]:
# all the data is now in a beautiful format 
# we nee a single tag for the vectorization so ,
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew'] + movies['keywords']

In [ ]:
movies.head(4)

In [ ]:
# let's make a new movies datframe
new_movies = movies[['movie_id','title','tags']]
new_movies

In [ ]:
new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_movies.head(3)

In [ ]:
# lets see a particular row
new_movies['tags'][0]

In [ ]:
# let's convert the string in lower case 
new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower())

In [ ]:
new_movies['tags'][563]

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
# root word
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_movies['tags'] = new_movies['tags'].apply(stem)

In [ ]:
stem("in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction samworthington zoesaldana sigourneyweaver jamescameron cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d")

In [ ]:
# model traning 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',max_features=5000)

In [ ]:
vectors = cv.fit_transform(new_movies['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors.shape

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out().tolist()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
similarity[1]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_movies[new_movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_movies.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle
pickle.dump(new_movies.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))